# Inicializar

In [1]:
import gurobipy as gp
from gurobipy import GRB
import os
import re
import pandas as pd

In [2]:
# Mostrar el modelo
def displayModel(model,name):
    try:
        os.remove(name)
    except OSError:
        pass
    model.write(name) # Escribir el modelo a un archivo .lp
    model.display()

In [3]:
L = 10
I = range(3) # 0: Low, 1: Medium, 2: High
c_values = {0: 5, 1: 12, 2: 20}

In [4]:
def mostrarResultadosTablaHibrido(cantidadNodos, costo, variablesDecision):

	if costo is None:
		print("====================================================")
		print("Cantidad de Nodos: ", cantidadNodos)
		print("====================================================")
		print("No se encontró solución")
	else:
		print("====================================================")
		print("Cantidad de Nodos: ", cantidadNodos)
		print("====================================================")
		print("Resultado de la Optimización:")
		print("====================================================")
		print("Costo Total: ", costo)
		print("Costo nodos: "+str(variablesDecision["nodesCost"]))
		print("Costo enlaces: "+str(variablesDecision["linksCost"]))
		print("====================================================")
		print("Nodos activos: \n")

		xVars = dict()
		yVars = dict()
		for var in variablesDecision.keys():
			if var.startswith("x"):
				xVars[var] = variablesDecision[var]
			if var.startswith("y"):
				yVars[var] = variablesDecision[var]

		xactiveNodes = []
		yactiveNodes = []
		for u in range(cantidadNodos):
			valoresx = []
			valoresy = []
			patternx = re.compile(r'^x\[' + str(u) + r',')
			patterny = re.compile(r'^y\[' + str(u) + r',')
			for var in xVars.keys():
				if patternx.match(var):
					valoresx.append(int(xVars[var]))
			for var in yVars.keys():
				if patterny.match(var):
					valoresy.append(int(yVars[var]))
			xactiveNodes.append(valoresx)
			yactiveNodes.append(valoresy)

		columns_titles_x = ["Low Cost", "Mid Cost", "High Cost"]
		columns_titles_y = [f"Subred {i}" for i in range(yactiveNodes[0].__len__())]
		row_index = [u+1 for u in range(cantidadNodos)]

		tablax = pd.DataFrame(
			xactiveNodes, columns=columns_titles_x, index=row_index)
		tablay = pd.DataFrame(
			yactiveNodes, columns=columns_titles_y, index=row_index)
		print(tablax)
		print("====================================================")
		print(tablay)
		print("====================================================")

def mostrarResultadosTabla(cantidadNodos, costo, variablesDecision):

	if costo is None:
		print("====================================================")
		print("Cantidad de Nodos: ", cantidadNodos)
		print("====================================================")
		print("No se encontró solución")
	else:
		print("====================================================")
		print("Cantidad de Nodos: ", cantidadNodos)
		print("====================================================")
		print("Resultado de la Optimización:")
		print("====================================================")
		print("Costo Total: ", costo)
		print("Costo nodos: "+str(variablesDecision["nodesCost"]))
		print("Costo enlaces: "+str(variablesDecision["linksCost"]))
		print("====================================================")
		print("Nodos activos: \n")

		xVars = dict()
		for var in variablesDecision.keys():
			if var.startswith("x"):
				xVars[var] = variablesDecision[var]

		xactiveNodes = []
		for u in range(cantidadNodos):
			valoresx = []
			patternx = re.compile(r'^x\[' + str(u) + r',')
			for var in xVars.keys():
				if patternx.match(var):
					valoresx.append(int(xVars[var]))
			xactiveNodes.append(valoresx)

		columns_titles_x = ["Low Cost", "Mid Cost", "High Cost"]
		row_index = [u+1 for u in range(cantidadNodos)]

		tablax = pd.DataFrame(
			xactiveNodes, columns=columns_titles_x, index=row_index)
		print(tablax)
		print("====================================================")

# Modelo base

In [5]:
def base_model(N):

	if N < 4:
		raise ValueError("El número de nodos debe ser al menos 4.")

	model = gp.Model(f"General_Model_{N}_Nodos")

	# Parámetros
	U = range(N) # Nodos a desplegar

	#####################################################################

	# Variables de decisión
	x = model.addVars(U, I, vtype=GRB.BINARY, name="x") # Variable de decisión para el tipo de nodos

	# Variables auxiliares
	nodesCost = model.addVar(vtype=GRB.CONTINUOUS, name="nodesCost") # Costo de los nodos desplegados
	model.addConstr(nodesCost == gp.quicksum(c_values[i] * x[u, i] for i in I for u in U), name="NodesCost_def")

	linksCost = model.addVar(vtype=GRB.CONTINUOUS, name="linksCost")
	model.addConstr(linksCost == 0, name="LinksCost_General") # Costo de los enlaces para el modelo general 0

	#####################################################################

	# Restricción de unicidad de nodos: cada nodo puede ser de un solo tipo
	model.addConstrs(
		(gp.quicksum(x[u, i] for i in I) == 1 for u in U), name="Unicidad_i"
	)

	#####################################################################

	model.setParam('OutputFlag', 0)  # No mostrar salida por defecto de Gurobi
	model.setObjective(nodesCost+linksCost, GRB.MINIMIZE)

	model.optimize()

	return model

# Modelos heredados

In [6]:
def serie_model(model_base, N):

	# Copiar el modelo base
	model = model_base.copy()

	############################################################

	# Recuperar variables
	x = { tuple(map(int, var.varName.split('[')[1].split(']')[0].split(','))): var for var in model.getVars() if "x" in var.varName }

	linksCost = model.getVarByName("linksCost")
	if linksCost is None:
		raise ValueError("No se encontró la variable de costo de enlaces.")

	############################################################

	# Actualizar restricciones de costos de enlaces
	linksCost_Condition = model.getConstrByName("LinksCost_General")
	if linksCost_Condition is not None:
		model.remove(linksCost_Condition)

	model.addConstr(linksCost == L*(N-1), name="LinksCost_Serie") # Costo de los enlaces en serie

	# Optimizar el modelo
	model.optimize()

	if model.status == GRB.OPTIMAL:
		variablesDecision = {var.varName: var.x for var in model.getVars()}
		return model.objVal, variablesDecision
	else:
		raise Exception("No se encontró una solución óptima.")

In [7]:
def parallel_model(model_base, N):

	# Copiar el modelo base
	model = model_base.copy()

	#####################################################################

	# Recuperar variables

	x = { tuple(map(int, var.varName.split('[')[1].split(']')[0].split(','))): var for var in model.getVars() if "x" in var.varName }

	linksCost = model.getVarByName("linksCost")
	if linksCost is None:
		raise ValueError("No se encontró la variable de costo de enlaces.")

	######################################################################

	# Actualizar restricciones de costos de enlaces
	linksCost_Condition = model.getConstrByName("LinksCost_General")
	if linksCost_Condition is not None:
		model.remove(linksCost_Condition)

	model.addConstr(2*linksCost == L*(N-1)*(N), name="LinksCost_Serie") # Costo de los enlaces en serie


	# Optimizar el modelo
	model.optimize()

	if model.status == GRB.OPTIMAL:
		variablesDecision = {var.varName: var.x for var in model.getVars()}
		return model.objVal, variablesDecision
	else:
		raise Exception("No se encontró una solución óptima.")

In [8]:
def hibrid_model(model_base, N, L):

	if N < 4:
		raise ValueError("El número de nodos debe ser al menos 3.")

	U = range(N) # Nodos a desplegar
	J = range(N // 3 + 1) # Subredes a desplegar

	#################################################################

	# Copiar el modelo base
	model = model_base.copy() # Copiar el modelo base

	#############################################################

	y = model.addVars(U, J, vtype=GRB.BINARY, name="y") # Variable de decisión para el tipo de subredes

	linksCost = model.getVarByName("linksCost")
	if linksCost is None:
		raise ValueError("No se encontró la variable de costo de enlaces.")

	#####################################################################

	# Número de nodos en serie
	N_s = model.addVar(vtype=GRB.INTEGER, name="N_s")
	model.addConstr(N_s == gp.quicksum(y[u, 0] for u in U), name="Ns_def")

	# Número de nodos en paralelo
	N_p = model.addVar(vtype=GRB.INTEGER, name="N_p")
	model.addConstr(N_p == gp.quicksum(y[u, j] for u in U for j in J if j>0), name="Np_def")

	# Variable para indicar si la subred j >= 1 está activa
	alpha = model.addVars(J, vtype=GRB.BINARY, name="alpha")
	model.addConstrs((alpha[j] >= y[u, j] for u in U for j in J if j > 0), name="Activar_Subred")

	# Variable para indicar el numero de nodos en paralelo por subred
	p = model.addVars(J, vtype=GRB.INTEGER, name="p")
	model.addConstrs((p[j] == gp.quicksum(y[u, j] for u in U) for j in J if j > 0), name="p_def")

	# Variable para indicar el número de enlaces en paralelo por subred
	z = model.addVars(J, vtype=GRB.INTEGER, name="z")
	model.addConstrs((2*z[j] == p[j]*(p[j]-1) for j in J if j > 0), name="Quadratic_Exact_z")

	####################################################################

	# Restricción de unicidad de subredes: cada nodo pertenece a una sola subred
	model.addConstrs(
		(gp.quicksum(y[u, j] for j in J) == 1 for u in U), name="Unicidad_j"
	)

	# 🔴 Esta restricción solo aplica en el modelo híbrido
	model.addConstr(N_p >= 3, name="Np_Condition") # Número de nodos en paralelo mínimo

	# Restricción: Si la subred j >= 1 existe, debe tener al menos 3 nodos
	model.addConstrs(
		(gp.quicksum(y[u, j] for u in U) >= 3 * alpha[j] for j in J if j > 0),
		name="Subredes_Min_3"
	)

	# Actualizar restricciones de costos de enlaces
	linksCost_Condition = model.getConstrByName("LinksCost_General")
	if linksCost_Condition is not None:
		model.remove(linksCost_Condition)

	H = gp.quicksum(alpha[j] for j in J) - 1 # Número de subredes activas - 1
	B = gp.quicksum(z[j] for j in J if j > 0) # Número de enlaces en paralelo por subred
	model.addConstr(linksCost == L*(N_s + H + B), name="LinksCost_Hibrido")

	#####################################################################

	model.optimize()

	if model.status == GRB.OPTIMAL:
		variablesDecision = dict()
		for var in model.getVars():
			variablesDecision[var.varName] = var.x
		return model.objVal, variablesDecision, model
	else:
		raise Exception("No se encontró una solución óptima.")

# Pruebas

In [14]:
N = 10
modeloBase = base_model(N)

## Serie

In [15]:
try:
	costoSerie, variablesDecisionSerie = serie_model(modeloBase, N)
	print("Modelo serie")
	mostrarResultadosTabla(N, costoSerie, variablesDecisionSerie)
except Exception as e:
	print(f"Error: {e}")

Modelo serie
Cantidad de Nodos:  10
Resultado de la Optimización:
Costo Total:  140.0
Costo nodos: 50.0
Costo enlaces: 90.0
Nodos activos: 

    Low Cost  Mid Cost  High Cost
1          1         0          0
2          1         0          0
3          1         0          0
4          1         0          0
5          1         0          0
6          1         0          0
7          1         0          0
8          1         0          0
9          1         0          0
10         1         0          0


## Paralelo

In [16]:
try:
	costoParalelo, variablesDecisionParalelo = parallel_model(modeloBase, N)
	print("\nModelo solo paralelo")
	mostrarResultadosTabla(N, costoParalelo, variablesDecisionParalelo)
except Exception as e:
	print(f"Error: {e}")


Modelo solo paralelo
Cantidad de Nodos:  10
Resultado de la Optimización:
Costo Total:  500.0
Costo nodos: 50.0
Costo enlaces: 450.0
Nodos activos: 

    Low Cost  Mid Cost  High Cost
1          1         0          0
2          1         0          0
3          1         0          0
4          1         0          0
5          1         0          0
6          1         0          0
7          1         0          0
8          1         0          0
9          1         0          0
10         1         0          0


## Hibrido

In [17]:
try:
	costoHibrido, variablesDecisionHibrido, modeloHibrido = hibrid_model(modeloBase, N, L)
	print("Modelo híbrido")
	mostrarResultadosTablaHibrido(N, costoHibrido, variablesDecisionHibrido)
except Exception as e:
	print(f"Error: {e}")

Modelo híbrido
Cantidad de Nodos:  10
Resultado de la Optimización:
Costo Total:  150.0
Costo nodos: 50.0
Costo enlaces: 100.0
Nodos activos: 

    Low Cost  Mid Cost  High Cost
1          1         0          0
2          1         0          0
3          1         0          0
4          1         0          0
5          1         0          0
6          1         0          0
7          1         0          0
8          1         0          0
9          1         0          0
10         1         0          0
    Subred 0  Subred 1  Subred 2  Subred 3
1          1         0         0         0
2          1         0         0         0
3          1         0         0         0
4          1         0         0         0
5          1         0         0         0
6          1         0         0         0
7          1         0         0         0
8          0         0         1         0
9          0         0         1         0
10         0         0         1         0
